# 8 WorkFlow  Semillerio

### 8.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán enriquecer

#### 8.2  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab
*   Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab



*  Si usted eligió modalidad *Gerencial* entonces NO debe modificar nada, su liderazgo logró que otras trabajaron por usted
*  Si usted eligió modalidad de *Analista Junior*, entonces donde dice archivo="gerencial_competencia_2025.csv.gz"  lo debe cambiar por  archivo="analistajr_competencia_2025.csv.gz"
*  Si usted eligió modalidad *Analista Senior*,  ya se las ingenierá SIN preguntar !

In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/labo1r"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi


## 8.3  Workflow Semillerio

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sat Jul 19 14:17:57 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,653541,35.0,1439357,76.9,1383805,74.0
Vcells,1218215,9.3,8388608,64.0,1924957,14.7


In [3]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

Loading required package: data.table

Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.


Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, isOpen, nullfile, parse, use, warnings




#### Parametros
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

### Usted DEBE ambiar "PARAM experimento" cada vez que desee correr un nuevo experimento

In [4]:
PARAM <- list()
PARAM$semilla_primigenia <- 314159

PARAM$experimento <- "-genetico-0gen"
PARAM$dataset <- "analistasr_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [5]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("WF", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### 8.3.1   Preprocesamiento del dataset

#### 8.3.1.1  DT incorporar dataset

In [6]:
# lectura del dataset
dataset <- fread(paste0("/content/buckets/b1/datasets/", PARAM$dataset))

#### 8.3.1.2  CA  Catastrophe Analysis
Se intentan reparar las variables que para un mes están con todos los valores en cero.

El método que se utiliza es **Machine Learning** se asigna NA also valores, si ha leido bien, es la "anti imputación de valores faltantes"
<br> Usted podrá aplicar aquí otros métodos

In [7]:
dataset[ foto_mes==202006, internet:=NA]
dataset[ foto_mes==202006, mrentabilidad:=NA]
dataset[ foto_mes==202006, mrentabilidad_annual:=NA]
dataset[ foto_mes==202006, mcomisiones:=NA]
dataset[ foto_mes==202006, mactivos_margen:=NA]
dataset[ foto_mes==202006, mpasivos_margen:=NA]
dataset[ foto_mes==202006, mcuentas_saldo:=NA]
dataset[ foto_mes==202006, ctarjeta_visa_transacciones:=NA]
dataset[ foto_mes==202006, mtarjeta_visa_consumo:=NA]
dataset[ foto_mes==202006, mtarjeta_master_consumo:=NA]
dataset[ foto_mes==202006, ccallcenter_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]

#### 8.3.1.3  DR  Data Drifting
Se intenta corregir el data drifting, quizas ajustando por IPC ...
<br>Esta parte podrá ser abordada por todos los Analistas y también la Gerenciapero se decide pedagogicamente no incluirla en esta primer version para reducir la carga cognitiva

In [8]:
# sin codigo en esta primera version del workflow

#### 8.3.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [9]:
# esta funcion atributos presentes existe debido a que las modalidades poseen datasets con distinta cantidad de campos
atributos_presentes <- function( patributos )
{
  atributos <- unique( patributos )
  comun <- intersect( atributos, colnames(dataset) )

  return(  length( atributos ) == length( comun ) )
}

# el mes 1,2, ..12
if( atributos_presentes( c("foto_mes") ))
  dataset[, kmes := foto_mes %% 100]

# variable extraida de una tesis de maestria de Irlanda
if( atributos_presentes( c("mpayroll", "cliente_edad") ))
  dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]



In [ ]:
# Agregar variables de las generaciones
gen <- 20 #cantidad de generaciones

var_pool <- fread("./var_pool.txt")

for (i in (1:gen)){
  rgx <- paste0("^gen_",i,"_")
  variables_gen <- var_pool[grepl(rgx, var_pool$Feature)]
  for (i in 1:nrow(variables_gen)){
    var_name <- variables_gen[i, "Feature"][[1]]
    formula <- variables_gen[i, "complete_form"][[1]]
    dataset <- dataset[, (var_name) := eval(parse(text = formula))]
  }
  
}

In [11]:
# visualizo las columas del dataset a esta etapa
colnames(dataset)

[1] "numero_de_cliente"                   
  [2] "foto_mes"                            
  [3] "active_quarter"                      
  [4] "cliente_vip"                         
  [5] "internet"                            
  [6] "cliente_edad"                        
  [7] "cliente_antiguedad"                  
  [8] "mrentabilidad"                       
  [9] "mrentabilidad_annual"                
 [10] "mcomisiones"                         
 [11] "mactivos_margen"                     
 [12] "mpasivos_margen"                     
 [13] "cproductos"                          
 [14] "tcuentas"                            
 [15] "ccuenta_corriente"                   
 [16] "mcuenta_corriente_adicional"         
 [17] "mcuenta_corriente"                   
 [18] "ccaja_ahorro"                        
 [19] "mcaja_ahorro"                        
 [20] "mcaja_ahorro_adicional"              
 [21] "mcaja_ahorro_dolares"                
 [22] "cdescubierto_preacordado"            
 [23] "mcuentas_saldo"                      
 [24] "ctarjeta_debito"                     
 [25] "ctarjeta_debito_transacciones"       
 [26] "mautoservicio"                       
 [27] "ctarjeta_visa"                       
 [28] "ctarjeta_visa_transacciones"         
 [29] "mtarjeta_visa_consumo"               
 [30] "ctarjeta_master"                     
 [31] "ctarjeta_master_transacciones"       
 [32] "mtarjeta_master_consumo"             
 [33] "cprestamos_personales"               
 [34] "mprestamos_personales"               
 [35] "cprestamos_prendarios"               
 [36] "mprestamos_prendarios"               
 [37] "cprestamos_hipotecarios"             
 [38] "mprestamos_hipotecarios"             
 [39] "cplazo_fijo"                         
 [40] "mplazo_fijo_dolares"                 
 [41] "mplazo_fijo_pesos"                   
 [42] "cinversion1"                         
 [43] "minversion1_pesos"                   
 [44] "minversion1_dolares"                 
 [45] "cinversion2"                         
 [46] "minversion2"                         
 [47] "cseguro_vida"                        
 [48] "cseguro_auto"                        
 [49] "cseguro_vivienda"                    
 [50] "cseguro_accidentes_personales"       
 [51] "ccaja_seguridad"                     
 [52] "cpayroll_trx"                        
 [53] "mpayroll"                            
 [54] "mpayroll2"                           
 [55] "cpayroll2_trx"                       
 [56] "ccuenta_debitos_automaticos"         
 [57] "mcuenta_debitos_automaticos"         
 [58] "ctarjeta_visa_debitos_automaticos"   
 [59] "mttarjeta_visa_debitos_automaticos"  
 [60] "ctarjeta_master_debitos_automaticos" 
 [61] "mttarjeta_master_debitos_automaticos"
 [62] "cpagodeservicios"                    
 [63] "mpagodeservicios"                    
 [64] "cpagomiscuentas"                     
 [65] "mpagomiscuentas"                     
 [66] "ccajeros_propios_descuentos"         
 [67] "mcajeros_propios_descuentos"         
 [68] "ctarjeta_visa_descuentos"            
 [69] "mtarjeta_visa_descuentos"            
 [70] "ctarjeta_master_descuentos"          
 [71] "mtarjeta_master_descuentos"          
 [72] "ccomisiones_mantenimiento"           
 [73] "mcomisiones_mantenimiento"           
 [74] "ccomisiones_otras"                   
 [75] "mcomisiones_otras"                   
 [76] "cforex"                              
 [77] "cforex_buy"                          
 [78] "mforex_buy"                          
 [79] "cforex_sell"                         
 [80] "mforex_sell"                         
 [81] "ctransferencias_recibidas"           
 [82] "mtransferencias_recibidas"           
 [83] "ctransferencias_emitidas"            
 [84] "mtransferencias_emitidas"            
 [85] "cextraccion_autoservicio"            
 [86] "mextraccion_autoservicio"            
 [87] "ccheques_depositados"                
 [88] "mcheques_depositados"                
 [89] "ccheques_emitidos"                 

#### 8.3.1.4  FE_rf Feature Engineering de nuevas variables a partir de hojas de Random Forest

Esto se mostrará unicamente a la *modalidad Analista Sr*

In [12]:
# No se implementa Feature Engineering a partir de Random Forest

#### 8.3.1.5  FEhist Feature Engineering historico

El Fature Engineering Histórico es la etapa que más aporta a la ganancia final, ya que enriquece cada registro del dataset con su historia.

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1  =  valor actual - lag1
* lag2  lags de orden 2
* delta2  = valor actual - lag2


(*) Excepto para los campos  <numero_de_cliente,  foto_mes,  clase_ternaria>

In [13]:
# Feature Engineering Historico
# ordeno el dataset
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 2
dataset[,
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}


Verificacion de los campos recien creados

In [14]:
ncol(dataset)
colnames(dataset)

[1] 773

[1] "numero_de_cliente"                          
  [2] "foto_mes"                                   
  [3] "active_quarter"                             
  [4] "cliente_vip"                                
  [5] "internet"                                   
  [6] "cliente_edad"                               
  [7] "cliente_antiguedad"                         
  [8] "mrentabilidad"                              
  [9] "mrentabilidad_annual"                       
 [10] "mcomisiones"                                
 [11] "mactivos_margen"                            
 [12] "mpasivos_margen"                            
 [13] "cproductos"                                 
 [14] "tcuentas"                                   
 [15] "ccuenta_corriente"                          
 [16] "mcuenta_corriente_adicional"                
 [17] "mcuenta_corriente"                          
 [18] "ccaja_ahorro"                               
 [19] "mcaja_ahorro"                               
 [20] "mcaja_ahorro_adicional"                     
 [21] "mcaja_ahorro_dolares"                       
 [22] "cdescubierto_preacordado"                   
 [23] "mcuentas_saldo"                             
 [24] "ctarjeta_debito"                            
 [25] "ctarjeta_debito_transacciones"              
 [26] "mautoservicio"                              
 [27] "ctarjeta_visa"                              
 [28] "ctarjeta_visa_transacciones"                
 [29] "mtarjeta_visa_consumo"                      
 [30] "ctarjeta_master"                            
 [31] "ctarjeta_master_transacciones"              
 [32] "mtarjeta_master_consumo"                    
 [33] "cprestamos_personales"                      
 [34] "mprestamos_personales"                      
 [35] "cprestamos_prendarios"                      
 [36] "mprestamos_prendarios"                      
 [37] "cprestamos_hipotecarios"                    
 [38] "mprestamos_hipotecarios"                    
 [39] "cplazo_fijo"                                
 [40] "mplazo_fijo_dolares"                        
 [41] "mplazo_fijo_pesos"                          
 [42] "cinversion1"                                
 [43] "minversion1_pesos"                          
 [44] "minversion1_dolares"                        
 [45] "cinversion2"                                
 [46] "minversion2"                                
 [47] "cseguro_vida"                               
 [48] "cseguro_auto"                               
 [49] "cseguro_vivienda"                           
 [50] "cseguro_accidentes_personales"              
 [51] "ccaja_seguridad"                            
 [52] "cpayroll_trx"                               
 [53] "mpayroll"                                   
 [54] "mpayroll2"                                  
 [55] "cpayroll2_trx"                              
 [56] "ccuenta_debitos_automaticos"                
 [57] "mcuenta_debitos_automaticos"                
 [58] "ctarjeta_visa_debitos_automaticos"          
 [59] "mttarjeta_visa_debitos_automaticos"         
 [60] "ctarjeta_master_debitos_automaticos"        
 [61] "mttarjeta_master_debitos_automaticos"       
 [62] "cpagodeservicios"                           
 [63] "mpagodeservicios"                           
 [64] "cpagomiscuentas"                            
 [65] "mpagomiscuentas"                            
 [66] "ccajeros_propios_descuentos"                
 [67] "mcajeros_propios_descuentos"                
 [68] "ctarjeta_visa_descuentos"                   
 [69] "mtarjeta_visa_descuentos"                   
 [70] "ctarjeta_master_descuentos"                 
 [71] "mtarjeta_master_descuentos"                 
 [72] "ccomisiones_mantenimiento"                  
 [73] "mcomisiones_mantenimiento"                  
 [74] "ccomisiones_otras"                          
 [75] "mcomisiones_otras"                          
 [76] "cforex"                                     
 [77] "cforex_buy"                                

#### 8.3.1.6  FEhist Reduccion dimensionalidad con canaritos

Esta etapa solo se mostrará a la *modalidad Anlista Sr* por algun canal secreto de forma de no confundir a los *Analista Jr*  nni distraer con detalles operativos a la estratégica *Modalidad Gerencial*

In [15]:
# No se implementa la reduccion de la dimensionalidad con canaritos

### 8.3.2 Modelado

#### 8.3.2.1 Training Strategy

Esta etapa de Workflow de  Training Strategy esta pensada para la *Modalidad Gerencial* que posee el dataset de [202005, 202109]
<br> Si usted es un Analista, posee el periodo de [201901, 202109] y deberá experimentar en que meses le conviene experimentar

<br> A la *Modalidad Gerencial* no se le complicada la vida con el undersampling de los continua, por eso PARAM$trainingstrategy$training_pct <- 1.0
<br> Sin embargo, si usted es  *Analista SR* posee un dataset 50 veces ( filas x columnas) más grande que la *Modalidad Gerencial*  y por un tema de velocidad y experimentación más rápida puede llegar a necesitar activar el undersampling de la clase mayoritaria, a pesar de estar corriendo en Google Cloud.

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202109  obviamente completo

* final_train =  [ 202005, 202107 ]  con un undersampling de los CONTINUA del 2% por un tema de velocidad

* training
   * testing = NO HAY
   * validation =  202107   completo, sin undersampling
   * training = [ 202005, 202106 ]  donde se consideran el 100% de los CONTINUA

In [16]:
PARAM$trainingstrategy$validate <- c(202105, 202104)

PARAM$trainingstrategy$training <- c(
  202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005, 202004, 202003
)

PARAM$trainingstrategy$training_pct <- 0.1


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [17]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

In [18]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

In [19]:
# preparo para que se puede hacer undersampling de los CONTINUA
#  solamente por un tema de VELOCIDAD
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]

# undersampling de los CONTINUA
dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
    (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < PARAM$trainingstrategy$training_pct ) ]


if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

dtrain <- lgb.Dataset(
  data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label= dataset[fold_train == TRUE, clase01],
  free_raw_data= TRUE
)

Loading required package: lightgbm



In [20]:
# datos de validation
dvalidate <- lgb.Dataset(
  data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
  label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
  free_raw_data= TRUE
)

nrow(dvalidate)

[1] 327335

####  8.3.2.2. Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve

es muy importante notar que intencionalmente  **NO** se está optimizando la funcion de ganancia del problema

* Cantidad de iteraciones inteligentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31 , Alienigenas Ancestrales contruyeron las pirámides y dejaron a la humanidad en un jeroglifico  *max_bin=31*
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 256]
  * min_data_in_leaf  [8, 8192]

In [21]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers


Attaching package: ‘ParamHelpers’


The following object is masked from ‘package:R.utils’:

    isVector



Attaching package: ‘mlr’


The following objects are masked from ‘package:R.utils’:

    resample, setThreshold


Loading required package: smoof

Loading required package: checkmate


Attaching package: ‘checkmate’


The following object is masked from ‘package:DiceKriging’:

    checkNames


The following object is masked from ‘package:R.utils’:

    asInt



Attaching package: ‘smoof’


The following objects are masked from ‘package:R.oo’:

    getDescription, getName




Definición de la Bayesian Optimization
<br> Si se desea optimizar un hiperparámetro que esta como fijo, debe QUITARSE de param_fijos y agregarse a PARAM$hipeparametertuning$hs

In [22]:
# valor ridiculamente bajo para que corra rapido en el aula y no molestar a la *Modalidad Gerencial*
PARAM$hipeparametertuning$num_interations <- 100

# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  extra_trees = FALSE,

  max_depth = -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split = 0.0, # min_gain_to_split >= 0.0
  min_sum_hessian_in_leaf = 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1 = 0.0, # lambda_l1 >= 0.0
  lambda_l2 = 0.0, # lambda_l2 >= 0.0

  bagging_fraction = 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction = 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction = 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance = FALSE, #
  scale_pos_weight = 1.0, # scale_pos_weight > 0.0

  drop_rate = 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop = 50, # <=0 means no limit
  skip_drop = 0.5, # 0.0 <= skip_drop <= 1.0

  max_bin= 31,
  num_iterations= 2048,  # valor grande, lo limita early_stopping_rounds
  early_stopping_rounds= 200
)

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeNumericParam("learning_rate", lower = 0.02, upper = 0.3),
  makeNumericParam("feature_fraction", lower = 0.05, upper = 0.90),
  makeNumericParam("coverage", lower = 0.05, upper = 1.0), # nuevo
  makeNumericParam("leaf_size", lower = 0.001, upper = 0.2) # nuevo
)

Función "señora caja negra"  que es llamada para verificar la realidad por la Bayesian Optimization

In [23]:
# En  x llegan los parmaetros de la bayesiana
#  devuelve la AUC en validate del modelo entrenado
#  en el parametro x llegan los hiperparámetros que se estan optimizando

EstimarGanancia_AUC_lightgbm <- function(x) {

  message(format(Sys.time(), "%a %b %d %X %Y"))

  # uno la lista de hiperparametros : fijos + variables
  param_completo <- c(PARAM$lgbm$param_fijos, x)

  # hago la transformacion de leaf_size y  coverage
  if( "leaf_size"  %in% names(param_completo) &
      "coverage"  %in% names(param_completo)
  )
  {
    # primero defino el tamaño de las hojas
    param_completo$min_data_in_leaf <- pmax( 1,  round( nrow(dtrain) * param_completo$leaf_size )  )
    # luego la cantidad de hojas en funcion del valor anterior, el coverage, y la cantidad de registros
    param_completo$num_leaves <- pmin( 131072,
      pmax( 8,  round( ( param_completo$coverage * nrow( dtrain ) / param_completo$min_data_in_leaf ) ) ))
  }

  if( "leaf_size"  %in% names(param_completo) &
      !( "coverage"  %in% names(param_completo) )
  )
  {
    # primero defino el tamaño de las hojas
    param_completo$min_data_in_leaf <- pmax( 1,  round( nrow(dtrain) * param_completo$leaf_size )  )
  }


  # entreno LightGBM
  modelo_train <- lgb.train(
    data= dtrain,
    valids= list(valid = dvalidate),
    eval= "auc",
    param= param_completo,
    verbose= -100
  )

  # recupero la AUC en validation
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # esta es la forma de devolver un parametro extra
  attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

  # hago espacio en la memoria
  rm(modelo_train)
  gc(full= TRUE, verbose= FALSE)

  return(AUC)
}

seteo de la Bayesian Optimization (complejo)
<br> copiado y pegado de la documentación de la librería

In [24]:
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
    fn= EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
    minimize= FALSE, # estoy Maximizando AUC
    noisy= FALSE,
    par.set= PARAM$hipeparametertuning$hs,
    has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
    save.on.disk.at.time= 600,
    save.file.path= "HT.RDATA"
)

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
    ctrl,
    iters= PARAM$hipeparametertuning$num_interations  # cantidad de iteraciones inteligentes
)

# defino el método estandar para la creacion de los puntos iniciales
#   los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# mas configuraciones
surr.km <- makeLearner(
    "regr.km",
    predict.type= "se",
    covtype= "matern3_2",
    control= list(trace = TRUE)
)

Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado
<br> por favor no se asuste con los warnings que pudieran aparecer

Si corrío a medias y llegó a las iteraciones inteligentes, en el archivo binario HT.RDATA quedó lo ya procesado y es utilizado para retomar la corrida desde lo último que llegó a grabar.

In [25]:
# inicio la optimizacion bayesiana

if (!file.exists("HT.RDATA")) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue("HT.RDATA") # retomo en caso que ya exista
}

Computing y column(s) for design. Not provided.

Sat Jul 19 14:22:09 2025

Sat Jul 19 14:24:02 2025

Sat Jul 19 14:24:44 2025

Sat Jul 19 14:25:41 2025

Sat Jul 19 14:26:35 2025

Sat Jul 19 14:27:32 2025

Sat Jul 19 14:28:14 2025

Sat Jul 19 14:29:36 2025

Sat Jul 19 14:33:33 2025

Sat Jul 19 14:35:26 2025

Sat Jul 19 14:38:28 2025

Sat Jul 19 14:43:03 2025

Sat Jul 19 14:43:32 2025

Sat Jul 19 14:43:56 2025

Sat Jul 19 14:45:42 2025

Sat Jul 19 14:48:58 2025

[mbo] 0: learning_rate=0.262; feature_fraction=0.385; coverage=0.867; leaf_size=0.143 : y = 0.922 : 112.9 secs : initdesign

[mbo] 0: learning_rate=0.215; feature_fraction=0.481; coverage=0.699; leaf_size=0.0147 : y = 0.927 : 42.3 secs : initdesign

[mbo] 0: learning_rate=0.104; feature_fraction=0.693; coverage=0.249; leaf_size=0.121 : y = 0.925 : 56.6 secs : initdesign

[mbo] 0: learning_rate=0.273; feature_fraction=0.0942; coverage=0.567; leaf_size=0.111 : y = 0.922 : 54.7 secs : initdesign

[mbo] 0: learning_rate=0.139; featur

la bayesian optimization ha corrido, extraigo los mejores hiperparametros

In [26]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y, -num_iterations)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file="BO_log.txt",
  sep="\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  list(learning_rate, feature_fraction, coverage, leaf_size)
]

print(PARAM$out$lgbm$mejores_hiperparametros)

   learning_rate feature_fraction  coverage  leaf_size
           <num>            <num>     <num>      <num>
1:    0.02034708         0.892623 0.9816102 0.03860156


### 8.3.3 Produccion Semillerio

### Final Training Semillerio
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

#### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [27]:
PARAM$trainingstrategy$final_train <- c( 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007, 202006, 202005, 202004, 202003
)

dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

# creo el dfinal_train en formato  LightGBM
dfinal_train <- lgb.Dataset(
  data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
  label= dataset[fold_final_train == TRUE, clase01],
  free_raw_data= TRUE
)

nrow( dfinal_train) # verifico el tamaño

[1] 2365834

#### Final Training Hyperparameters

In [28]:
# uno los parametros fijos y los mejores encontrados de los variables
fijos <- copy(PARAM$lgbm$param_fijos)

# quito lo que optimice en la Bayesian Optimization
fijos$num_iterations <- NULL
fijos$early_stopping_rounds <- NULL

# agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

# hago la transformacion de leaf_size y  coverage
  if( "leaf_size"  %in% names(param_final) &
      "coverage"  %in% names(param_final)
  )
  {
    # primero defino el tamaño de las hojas
    param_final$min_data_in_leaf <- pmax( 1,  round( nrow(dfinal_train) * param_final$leaf_size )  )
    # luego la cantidad de hojas en funcion del valor anterior, el coverage, y la cantidad de registros
    param_final$num_leaves <- pmin( 131072,
      pmax( 8,  round( ( param_final$coverage * nrow( dfinal_train ) / param_final$min_data_in_leaf ) ) ))
  }

  if( "leaf_size"  %in% names(param_final) &
      !( "coverage"  %in% names(param_final) )
  )
  {
    # primero defino el tamaño de las hojas
    param_final$min_data_in_leaf <- pmax( 1,  round( nrow(dfinal_train) * param_final$leaf_size )  )
  }

In [29]:
param_final

$objective
[1] "binary"

$metric
[1] "auc"

$first_metric_only
[1] TRUE

$boost_from_average
[1] TRUE

$feature_pre_filter
[1] FALSE

$verbosity
[1] -100

$force_row_wise
[1] TRUE

$seed
[1] 314159

$extra_trees
[1] FALSE

$max_depth
[1] -1

$min_gain_to_split
[1] 0

$min_sum_hessian_in_leaf
[1] 0.001

$lambda_l1
[1] 0

$lambda_l2
[1] 0

$bagging_fraction
[1] 1

$pos_bagging_fraction
[1] 1

$neg_bagging_fraction
[1] 1

$is_unbalance
[1] FALSE

$scale_pos_weight
[1] 1

$drop_rate
[1] 0.1

$max_drop
[1] 50

$skip_drop
[1] 0.5

$max_bin
[1] 31

$learning_rate
[1] 0.02034708

$feature_fraction
[1] 0.892623

$coverage
[1] 0.9816102

$leaf_size
[1] 0.03860156

$min_data_in_leaf
[1] 91325

$num_leaves
[1] 25

#### Training  Semillerio
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

### Aqui va cuantas semillas utilizo para el semillerio

In [30]:
# semillerio
PARAM$FT$semillerio <- 20  # cantidad de semillas

In [31]:
# semillerio
if(!require("primes")) install.packages("primes")
require("primes")

Loading required package: primes



In [32]:
# semillerio
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
# me quedo con PARAM$semillerio  primos al azar
PARAM$FT$semillas <- sample(primos)[seq(PARAM$FT$semillerio)]

cat( PARAM$FT$semillas)

303491

In [33]:
# semillerio
dir.create("modelos", showWarnings =FALSE)

primero <- TRUE
for( sem in PARAM$FT$semillas)
{
  nombre_arch <- paste0( "./modelos/modelo_", sem, ".txt")
  if( !file.exists(nombre_arch) )
  {
    param_final$seed <- sem

    set.seed(sem, kind = "L'Ecuyer-CMRG")
    final_model <- lgb.train(
      data= dfinal_train,
      param= param_final,
      verbose= -100
    )

    lgb.save(final_model, nombre_arch) # grabo el modelo"

    # grabo la primer importancia de variables
    #  Natalia : da lo mismo cual se guarda
    if( primero)
    {
      primero <- FALSE
      tb_importancia <- as.data.table(lgb.importance(final_model))
      archivo_importancia <- "impo.txt"

      fwrite( tb_importancia,
        file= archivo_importancia,
        sep= "\t"
      )
    }
 }
}

### Scoring Semillerio

Aplico el modelo final a los datos del futuro

In [34]:
PARAM$trainingstrategy$future <- c(202107)

dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

In [35]:
# semillerio aplico final_model   a dfuture

tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := 0]

datos_matrix <- data.matrix(dfuture[, campos_buenos, with= FALSE])


for( isem in seq(length(PARAM$FT$semillas)) )
{
   sem <- PARAM$FT$semillas[ isem ]
   nombre_arch <- paste0( "./modelos/modelo_", sem, ".txt")
   final_model <- lgb.load(nombre_arch)

   prediccion <- predict(
     final_model,
     datos_matrix
  )

  tb_prediccion[, paste0("prob_", isem) := prediccion]
  tb_prediccion[, prob := prob + prediccion]

  rm(final_model)
  rm(prediccion)
  gc(full = TRUE, verbose=FALSE)
}

rm( datos_matrix)
gc(full = TRUE, verbose=FALSE)

tb_prediccion[, prob := prob /length(PARAM$FT$semillas) ]

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2438463,130.3,4819114,257.4,4819114,257.4
Vcells,2930046900,22354.5,7223430399,55110.5,6932409275,52890.1


In [36]:
tb_prediccion

numero_de_cliente,prob,prob_1
<int>,<dbl>,<dbl>
29183733,0.001254344,0.001254344
29184468,0.001247419,0.001247419
29185245,0.018101841,0.018101841
29186441,0.006990371,0.006990371
29186475,0.006565154,0.006565154
29187730,0.001534016,0.001534016
29187764,0.001498767,0.001498767
29187961,0.003256082,0.003256082
29189899,0.003675222,0.003675222


In [37]:
# Agregamos la clase real al tb_prediccion para poder evaluar wilcoxon luego
tb_prediccion <- tb_prediccion[, clase_ternaria := dfuture$clase_ternaria] 

In [38]:
# grabo las probabilidad del modelo
#  me va a ser util para hacer Ensembles de modelos

fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

In [39]:
# inicializamos data.table
tb_ganancias <- data.table(isem = integer(), max_gain = numeric())

# loop para maximos de ganancia
for (isem in seq(length(PARAM$FT$semillas))){
    # cliente | prob_isem | clase
    pred <- tb_prediccion[, c("numero_de_cliente",  paste0("prob_", isem), "clase_ternaria"), with = FALSE]

    # Ordenar por probabilidad descendente
    pred <- pred[order(-get(paste0("prob_", isem)))]

    # Calcular ganancia y ganancia acumulada
    pred[, Ganancia := ifelse(clase_ternaria == "BAJA+2", 117000, -3000)]
    pred[, Ganancia_acum := cumsum(Ganancia)]

    # Seleccionar ciertas filas
    pred <- pred[seq(6000, 14000, 100),]

    # Seleccionamos el máximo
    max_gain <- max(pred[, Ganancia_acum])

    tb_ganancias <- rbind(tb_ganancias, data.table(isem = isem, max_gain = max_gain))

}

fwrite(tb_ganancias,
  file= "tb_ganancias.txt",
  sep= "\t"
)

In [40]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sat Jul 19 14:54:47 2025"